In [1]:
%reload_ext autoreload
%autoreload 2

In [1]:
from depsurf import BuildVersion

BuildVersion.filter(arch="amd64", version="6.8.0")[0].img.tracepoints

[   symtab.py:33 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/6.8.0-22-generic-amd64.pkl
[      btf.py:16 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/6.8.0-22-generic-amd64.pkl


Tracepoints: ['initcall_finish', 'initcall_start', 'initcall_level', 'emulate_vsyscall', 'xen_cpu_set_ldt', 'xen_cpu_write_gdt_entry', 'xen_cpu_load_idt', 'xen_cpu_write_idt_entry', 'xen_cpu_write_ldt_entry', 'xen_mmu_write_cr3', 'xen_mmu_flush_tlb_multi', 'xen_mmu_flush_tlb_one_user', 'xen_mmu_pgd_unpin', 'xen_mmu_pgd_pin', 'xen_mmu_release_ptpage', 'xen_mmu_alloc_ptpage', 'xen_mmu_ptep_modify_prot_commit', 'xen_mmu_ptep_modify_prot_start', 'xen_mmu_set_p4d', 'xen_mmu_set_pud', 'xen_mmu_set_pmd', 'xen_mmu_set_pte', 'xen_mc_extend_args', 'xen_mc_flush', 'xen_mc_flush_reason', 'xen_mc_callback', 'xen_mc_entry_alloc', 'xen_mc_entry', 'xen_mc_issue', 'xen_mc_batch', 'hyperv_send_ipi_one', 'hyperv_send_ipi_mask', 'hyperv_nested_flush_guest_mapping_range', 'hyperv_nested_flush_guest_mapping', 'hyperv_mmu_flush_tlb_multi', 'vector_free_moved', 'vector_setup', 'vector_teardown', 'vector_deactivate', 'vector_activate', 'vector_alloc_managed', 'vector_alloc', 'vector_reserve', 'vector_reserve_m

In [8]:
from itertools import pairwise

from depsurf import BuildVersion, FileLogger, check_diff, diff_dict, OUTPUT_PATH

versions = BuildVersion.filter(flavor="generic", arch="amd64")

total_struct_only = 0
total_func_only = 0
total_both = 0

results = {}

for (v1, v2) in list(pairwise(versions)):
    print(f"Comparing {v1} and {v2}")

    tp1 = v1.img.tracepoints
    tp2 = v2.img.tracepoints
    with FileLogger(path=None):
        func_changed = check_diff(tp1.funcs, tp2.funcs).changed
        struct_changed = check_diff(tp1.events, tp2.events).changed

        struct_only, func_only, both = diff_dict(func_changed, struct_changed)

        total_struct_only += len(struct_only)
        total_func_only += len(func_only)
        total_both += len(both)

        for k in list(struct_only) + list(func_only) + list(both):
            results[k] = results.get(k, 0) + 1

    with FileLogger(path=OUTPUT_PATH / "tp" / f"{v1}-{v2}.txt"):
        for (d, name) in [(struct_only, "Struct only"), (func_only, "Func only"), (both, "Both")]:
            print(f"{name}: {len(d)}")
            for k, v in d.items():
                print(f"\t{k}")
                if name == "Both":
                    v1, v2 = v
                    v1.print(nindent=2)
                    v2.print(nindent=2)
                else:
                    v.print(nindent=2)

Comparing 4.4.0-21-generic-amd64 and 4.8.0-22-generic-amd64
Comparing 4.8.0-22-generic-amd64 and 4.10.0-19-generic-amd64
Comparing 4.10.0-19-generic-amd64 and 4.13.0-16-generic-amd64
Comparing 4.13.0-16-generic-amd64 and 4.15.0-20-generic-amd64
Comparing 4.15.0-20-generic-amd64 and 4.18.0-10-generic-amd64
Comparing 4.18.0-10-generic-amd64 and 5.0.0-13-generic-amd64
Comparing 5.0.0-13-generic-amd64 and 5.3.0-18-generic-amd64
Comparing 5.3.0-18-generic-amd64 and 5.4.0-26-generic-amd64
Comparing 5.4.0-26-generic-amd64 and 5.8.0-25-generic-amd64
Comparing 5.8.0-25-generic-amd64 and 5.11.0-16-generic-amd64
Comparing 5.11.0-16-generic-amd64 and 5.13.0-19-generic-amd64
Comparing 5.13.0-19-generic-amd64 and 5.15.0-25-generic-amd64
Comparing 5.15.0-25-generic-amd64 and 5.19.0-21-generic-amd64
Comparing 5.19.0-21-generic-amd64 and 6.2.0-20-generic-amd64
Comparing 6.2.0-20-generic-amd64 and 6.5.0-9-generic-amd64
Comparing 6.5.0-9-generic-amd64 and 6.8.0-22-generic-amd64


In [9]:
print(total_func_only)
print(total_struct_only)
print(total_both)

55
234
124


In [10]:
for k, v in sorted(results.items(), key=lambda x: x[1], reverse=True):
    print(f"{k}: {v}")

mm_vmscan_lru_isolate: 5
mm_vmscan_lru_shrink_inactive: 4
fib6_table_lookup: 4
fib_table_lookup: 4
mm_vmscan_memcg_softlimit_reclaim_begin: 3
mm_vmscan_memcg_reclaim_begin: 3
mm_vmscan_direct_reclaim_begin: 3
x86_fpu_xstate_check_failed: 3
x86_fpu_copy_dst: 3
x86_fpu_copy_src: 3
x86_fpu_dropped: 3
x86_fpu_init_state: 3
x86_fpu_regs_deactivated: 3
x86_fpu_regs_activated: 3
x86_fpu_after_restore: 3
x86_fpu_before_restore: 3
x86_fpu_after_save: 3
x86_fpu_before_save: 3
io_uring_complete: 3
xhci_hub_status_data: 3
xhci_get_port_status: 3
xhci_handle_port_status: 3
io_uring_poll_arm: 3
io_uring_submit_sqe: 3
writeback_single_inode: 2
writeback_single_inode_start: 2
writeback_sb_inodes_requeue: 2
balance_dirty_pages: 2
bdi_dirty_ratelimit: 2
writeback_queue_io: 2
wbc_writepage: 2
writeback_wake_background: 2
writeback_wait: 2
writeback_written: 2
writeback_start: 2
writeback_exec: 2
writeback_queue: 2
writeback_write_inode: 2
writeback_write_inode_start: 2
mm_compaction_try_to_compact_pages: